# 仿真剖析：深入SWP核心引擎

欢迎来到本次对智能水务平台（SWP）核心的深入探讨。本Jupyter Notebook旨在解释驱动平台内任何仿真的基本组件。理解这些核心概念是构建、扩展和调试您自己的水系统仿真的关键。

我们将探讨任何仿真中的关键角色：
- **`仿真平台 (SimulationHarness)`**: 我们管弦乐队的中央指挥。
- **组件 (Components)**: 单个的乐器，代表如水库和闸门等物理对象。
- **代理 (Agents)**: 音乐家，代表感知和行动的自治实体。
- **`消息总线 (MessageBus)`**: 允许所有人进行交流的乐谱和提示。

## 第1节: `仿真平台 (SimulationHarness)` - 指挥家

`SimulationHarness` 是运行仿真最重要的对象。它充当中央管理器，负责：

- **持有**所有属于仿真的**组件和代理**。
- **管理仿真配置**，例如其持续时间 (`duration`) 和时间步长 (`dt`)。
- **运行主仿真循环**，按时间步进并协调所有部分之间的交互。
- **提供一个 `消息总线 (MessageBus)`**，供代理在不直接耦合的情况下进行通信。

## 第2节: 组件 - 管弦乐队

仿真中随时间变化的每个对象（如水库、闸门，甚至控制器）都是一个**组件 (Component)**。为确保它们都能协同工作，它们必须遵守一个契约。

这个契约由 `swp.core.interfaces` 中的 `Simulatable` 接口定义。任何 `Simulatable` 对象都必须有：

- 一个 `step(action, dt)` 方法：这是组件的核心。它接受一个 `action` (例如，新的闸门开启度) 和一个时间段 `dt`，并计算其新状态。
- 一个 `get_state()` 方法：这返回一个代表组件当前状态的字典 (例如, `{'volume': 1000, 'water_level': 10.5}`)。
- `参数 (Parameters)`: 每个组件都用一个定义其物理属性（例如，管道的长度和直径）的参数字典进行初始化。

## 第3节: 拓扑结构 - 连接各个部分

水系统是网络。一个水库流入一根管道，管道流入一个水轮机。`SimulationHarness` 需要理解这个我们称之为**拓扑 (topology)** 的物理网络。

我们通过以下方式构建拓扑：
1. 将每个组件添加到平台: `harness.add_component(...)`
2. 定义它们之间的有向链接: `harness.add_connection("upstream_component", "downstream_component")`

当调用 `harness.build()` 时，它会对这个网络执行**拓扑排序**。这会创建一个特定的更新顺序，以确保当我们计算下游组件的状态时，其上游源的状态已经为当前时间步计算完毕。这避免了循环依赖并确保了仿真的稳定性。

## 第4节: `消息总线 (MessageBus)` - 通信渠道

在一个简单的仿真中，平台可以直接控制组件。但在一个**多代理系统 (MAS)**中，组件和代理需要解耦。一个控制闸门的代理不应该需要对闸门对象的直接引用；它应该只发送一个意图改变闸门开启度的消息。

这是通过 `MessageBus` 实现的。它使用一个简单的**发布-订阅**模式：
- 一个代理可以向特定主题**发布**一条消息 (例如, `bus.publish("action.gate.opening", {'control_signal': 0.5})`)。
- 一个组件或另一个代理可以**订阅**该主题 (`bus.subscribe("action.gate.opening", self.handle_action_message)`)。当一条消息被发布到该主题时，订阅者的处理方法会被自动调用。

这保持了系统的模块化，并允许在不重写现有代码的情况下添加新的代理和组件。

## 第5节: 案例研究 - 一个简单的MAS控制循环

让我们把所有这些概念放在一起。下面的代码设置并运行一个完整的多代理仿真。

**场景**: 一个 `水库 (Reservoir)` 的水位需要维持在12.0m的设定点。出流量由一个 `闸门 (Gate)` 控制。

**多代理系统 (MAS)**:
1. 一个用于水库的 `数字孪生代理 (DigitalTwinAgent)` 感知其状态并将其 `water_level` **发布**到主题 `"state.reservoir.level"`。
2. 一个 `本地控制代理 (LocalControlAgent)` **订阅**这个主题。当它接收到新的水位时，其内部的 `PIDController` 计算一个新的期望闸门开启度。
3. 然后 `LocalControlAgent` 将这个新的开启度**发布**到主题 `"action.gate.opening"`。
4. `Gate` 本身**订阅**了这个动作主题，并相应地调整其开启度。

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.core_engine.testing.simulation_harness import SimulationHarness

# 1. 创建平台并获取消息总线
simulation_config = {'duration': 300, 'dt': 1.0}
harness = SimulationHarness(config=simulation_config)
message_bus = harness.message_bus

# 2. 定义通信主题
RESERVOIR_STATE_TOPIC = "state.reservoir.level"
GATE_ACTION_TOPIC = "action.gate.opening"

# 3. 创建物理组件
gate_params = {'max_rate_of_change': 0.1, 'discharge_coefficient': 0.6, 'width': 10, 'max_opening': 1.0}
reservoir = Reservoir(name="reservoir_1", initial_state={'volume': 21e6, 'water_level': 14.0}, parameters={'surface_area': 1.5e6})
# 注意: 闸门订阅了消息总线以获取其动作
gate = Gate(name="gate_1", initial_state={'opening': 0.1}, parameters=gate_params, message_bus=message_bus, action_topic=GATE_ACTION_TOPIC)

# 4. 创建代理
twin_agent = DigitalTwinAgent(agent_id="twin_agent_reservoir_1", simulated_object=reservoir, message_bus=message_bus, state_topic=RESERVOIR_STATE_TOPIC)
pid_controller = PIDController(Kp=-0.5, Ki=-0.01, Kd=-0.1, setpoint=12.0, min_output=0.0, max_output=gate_params['max_opening'])
control_agent = LocalControlAgent(agent_id="control_agent_gate_1", controller=pid_controller, message_bus=message_bus, observation_topic=RESERVOIR_STATE_TOPIC, observation_key='water_level', action_topic=GATE_ACTION_TOPIC, dt=harness.dt)

# 5. 将所有组件和代理添加到平台
harness.add_component(reservoir)
harness.add_component(gate)
harness.add_agent(twin_agent)
harness.add_agent(control_agent)

# 6. 定义物理拓扑
harness.add_connection("reservoir_1", "gate_1")

# 7. 构建并运行仿真
import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    harness.build()
    # 对基于代理的仿真使用 run_mas_simulation
    harness.run_mas_simulation()
sys.stdout = original_stdout

print("案例研究仿真完成。")

## 第6节: 可视化结果

现在仿真已经完成，我们可以使用 `harness.history` (我们在前一步中添加的) 来绘制结果。下面的图表显示了反馈控制系统的经典行为：控制器调节闸门开启度，以驱动水库的水位朝向期望的设定点12.0m。

In [ ]:
# 从历史记录中提取数据
time = [h['time'] for h in harness.history]
reservoir_levels = [h['reservoir_1']['water_level'] for h in harness.history]
gate_openings = [h['gate_1']['opening'] for h in harness.history]

# 创建一个DataFrame以便于查看和绘图
df = pd.DataFrame({
    'Time': time,
    'Reservoir Level': reservoir_levels,
    'Gate Opening': gate_openings
})

print("仿真历史的前5个步骤:")
print(df.head())

# 绘制结果
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

ax1.plot(df['Time'], df['Reservoir Level'], label='水库水位')
ax1.axhline(y=12.0, color='r', linestyle='--', label='设定点 (12.0m)')
ax1.set_ylabel('水位 (m)')
ax1.set_title('MAS反馈控制仿真')
ax1.grid(True)
ax1.legend()

ax2.plot(df['Time'], df['Gate Opening'], label='闸门开启度', color='purple')
ax2.set_xlabel('时间 (s)')
ax2.set_ylabel('闸门开启度 (%)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()

## 结论

您现在已经理解了智能水务平台的关键构建块。通过将 `SimulationHarness`、`Simulatable` 组件、物理 `topology` 和用于 `Agent` 通信的 `MessageBus` 结合起来，我们可以创建复杂、解耦且强大的水系统仿真。